In [288]:
import json
import time
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.microsoft.service import Service
from selenium.webdriver.common.by import By
# from create_txt_with_encoding import createFile


In [289]:
def launchBrowser(url):
    print('in launchBrowser')
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
    driver.get(url)
    return driver

In [290]:
def get_disease_list(driver):
    print('in get_disease_list')
    disease_list = []
    time.sleep(2)
    page = driver.find_element(By.XPATH, "/html/body/section[2]/div/div/div")
    find_results = page.find_elements(By.TAG_NAME, 'li')
    for i in find_results:
        disease_list.append(i.text)
    return disease_list

In [291]:
def get_paragraph(dictionary, disease, dict_key, next_element):
    print('in get_paragraph')
    dictionary[disease][dict_key] += next_element.text
    return dictionary[disease][dict_key]

In [437]:
def get_symptoms(dictionary, disease, next_element):
    print('in get_symptoms')
    
    if next_element.tag_name == 'p':
        dictionary[disease]["Symptoms"] = get_paragraph(dictionary, disease, "Symptoms", next_element)
    return dictionary[disease]["Symptoms"]

In [414]:
def getDiagnosis(dictionary, disease, next_element):
    print('in getDiagnonis')
    if next_element.tag_name == 'p':
        dictionary[disease]['Diagnosis'] = get_paragraph(dictionary, disease, "Diagnosis", next_element)
    return dictionary[disease]['Diagnosis']

In [515]:
def getRiskFactors(dictionary, disease, next_element):
    print('in getRiskFactors')
    if next_element.tag_name == 'p':
        dictionary[disease]['RiskFactors'] = get_paragraph(dictionary, disease, "RiskFactors", next_element)
    return dictionary[disease]['RiskFactors']

In [516]:
def getPrevention(dictionary, disease, next_element):
    print('in getPrevention')
    if next_element.tag_name == 'p':
        dictionary[disease]['Prevention'] = get_paragraph(dictionary, disease, "Prevention", next_element)
    return dictionary[disease]['Prevention']

In [539]:
def fetchData(driver, disease, dictionary, symptoms, diagnosis, risk_factors, prevention):
    print('in fetch data')
    container = driver.find_element(By.ID, "bodyContent")
    web_elements = container.find_elements(By.XPATH, ".//*")
    iter_elements = iter(web_elements)
    
    for el in range(0, len(web_elements)-1):
        try:
            next_element = next(iter_elements)
        except Exception as e:
            print('Exception', e)
            return dictionary[disease]
        if next_element.get_attribute('id'):
            if next_element.tag_name == 'span':
                print("element.get_attribute('id')", next_element.get_attribute('id'))
                if 'symptoms' in next_element.get_attribute('id').lower():
                    print('checking symptoms')
                    while next_element.tag_name != 'p':
                        try:
                            next_element = next(iter_elements)
                        except:
                            return dictionary[disease]
                    print(next_element.tag_name, next_element.text)
                    symptoms += get_symptoms(dictionary, disease, next_element)
                elif 'diagnosis' in next_element.get_attribute('id').lower():
                    print('checking diagnosis')
                    while next_element.tag_name != 'p':
                        try:
                            next_element = next(iter_elements)
                        except:
                            return dictionary[disease]
                    
                    diagnosis += getDiagnosis(dictionary, disease, next_element)
                 
                elif 'risk_factors' in next_element.get_attribute('id').lower():
                    while next_element.tag_name != 'p':
                        try:
                            next_element = next(iter_elements)
                        except:
                            return dictionary[disease]
                    risk_factors += getRiskFactors(dictionary, disease, next_element)
                elif 'prevention' in next_element.get_attribute('id').lower():
                    while next_element.tag_name != 'p':
                        try:
                            next_element = next(iter_elements)
                        except:
                            return dictionary[disease]
                    prevention += getPrevention(dictionary, disease, next_element)
                
                elif 'references' in next_element.get_attribute('id').lower():
                    break
                    
        
    return dictionary[disease]


In [540]:
def wikipedia_search(driver, link, disease_list):
    data_confirmation_dictionary ={}
    disease_details = {}
    
    print('in wikipedia search')
    for disease in disease_list:
        driver.get(link + disease)
        data_confirmation_dictionary[disease] = ''
        disease_details[disease] = {}

        disease_details[disease]["Symptoms"] = ''
        disease_details[disease]["Diagnosis"] = ''
        disease_details[disease]["RiskFactors"]  = ''
        disease_details[disease]["Prevention"]  = ''
        
        print('a',  disease_details)
        try:
            disease_data = driver.find_element(By.ID, 'Signs_and_symptoms')
            data_confirmation_dictionary[disease] = 'has data'
            disease_details[disease] = fetchData(driver, disease, disease_details, disease_details[disease]["Symptoms"],  disease_details[disease]["Diagnosis"], disease_details[disease]["RiskFactors"], disease_details[disease]["Prevention"])
            print('b',  disease_details)
        except Exception as e:
            print(e)
            data_confirmation_dictionary[disease] = 'no data found'
            print('c',  disease_details)
            pass         
        save_data(disease_details)
    print('e',  disease_details)
    return data_confirmation_dictionary, disease_details

In [541]:
def createFile(filename):
    with open(f'{filename}.json', 'w', encoding='utf-8') as f:
        f.close()

In [533]:
def save_data(disease_details):
    with open('DiseaseData.json', 'w', encoding='utf-8') as f:
        f.write(str(disease_details))

In [530]:
save_data(disease_details)

In [568]:

if __name__ == '__main__':
    #file = open("path_and_tags.json")
    #data = json.load(file)
    #disease_list_url = 'https://www.thehealthsite.com/list-of-diseases-and-conditions/'
    driver = launchBrowser(disease_list_url)
    #disease_list = get_disease_list(driver)
    data_cofirmation_dictionary, disease_details = wikipedia_search(driver, 'https://en.wikipedia.org/wiki/', disease_list)
    
    #print(disease_details)
    # save_data(disease_details)




in launchBrowser
in wikipedia search
a {'Congential heart disease': {'Symptoms': '', 'Diagnosis': '', 'RiskFactors': '', 'Prevention': ''}}
Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="Signs_and_symptoms"]"}
  (Session info: MicrosoftEdge=112.0.1722.64)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00007FF72E7EE0B2+60274]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF72E77ED12+818898]
	(No symbol) [0x00007FF72E3FDFAE]
	(No symbol) [0x00007FF72E43FAA1]
	(No symbol) [0x00007FF72E43FC28]
	(No symbol) [0x00007FF72E47B687]
	(No symbol) [0x00007FF72E45F5AF]
	(No symbol) [0x00007FF72E432751]
	(No symbol) [0x00007FF72E47891C]
	(No symbol) [0x00007FF72E45F343]
	(No symbol) [0x00007FF72E431796]
	(No symbol) [0x00007FF72E430975]
	(No symbol) [0x00007FF72E431F04]
	Microsoft::Applications::Events::EventProperties::SetLevel [0x00007FF72E687187+1678119]
	Microsoft::Applications::Events::EventProperties::SetLevel [0x00007FF72E

element.get_attribute('id') Signs_and_symptoms
checking symptoms
p Hepatitis has a broad spectrum of presentations that range from a complete lack of symptoms to severe liver failure.[17][18][19] The acute form of hepatitis, generally caused by viral infection, is characterized by constitutional symptoms that are typically self-limiting.[17][18] Chronic hepatitis presents similarly, but can manifest signs and symptoms specific to liver dysfunction with long-standing inflammation and damage to the organ.[19][20]
in get_symptoms
in get_paragraph
element.get_attribute('id') Acute_hepatitis
element.get_attribute('id') Fulminant_hepatitis
element.get_attribute('id') Chronic_hepatitis
element.get_attribute('id') Causes
element.get_attribute('id') Infectious
element.get_attribute('id') Viral_hepatitis
element.get_attribute('id') Parasitic_hepatitis
element.get_attribute('id') Bacterial_hepatitis
element.get_attribute('id') Metabolic
element.get_attribute('id') Alcoholic_hepatitis
element.get_

element.get_attribute('id') Signs_and_symptoms
checking symptoms
p Cirrhosis can take quite a long time to develop, and symptoms may be slow to emerge.[8] Some early symptoms include tiredness, weakness, loss of appetite, weight loss, and nausea.[8] People may also feel discomfort in the right upper abdomen around the liver.[8]
in get_symptoms
in get_paragraph
element.get_attribute('id') Liver_dysfunction
element.get_attribute('id') Portal_hypertension
element.get_attribute('id') Other_nonspecific_signs
element.get_attribute('id') Advanced_disease
element.get_attribute('id') Causes
element.get_attribute('id') Common_causes
element.get_attribute('id') Less_common_causes
element.get_attribute('id') Pathophysiology
element.get_attribute('id') Diagnosis
checking diagnosis
in getDiagnonis
in get_paragraph
element.get_attribute('id') Imaging
element.get_attribute('id') Lab_findings
element.get_attribute('id') Pathology
element.get_attribute('id') Grading
element.get_attribute('id') Preventio

element.get_attribute('id') Signs_and_symptoms
checking symptoms
p Signs and symptoms include high blood pressure, headaches, abdominal pain, blood in the urine, and excessive urination.[1] Other symptoms include pain in the back, and cyst formation (renal and other organs).[9]
in get_symptoms
in get_paragraph
element.get_attribute('id') Cause
element.get_attribute('id') Autosomal_dominant
element.get_attribute('id') Autosomal_recessive
element.get_attribute('id') Mechanism
element.get_attribute('id') Diagnosis
checking diagnosis
in getDiagnonis
in get_paragraph
element.get_attribute('id') Natural_history
element.get_attribute('id') Treatment
element.get_attribute('id') Prognosis
element.get_attribute('id') Epidemiology
element.get_attribute('id') See_also
element.get_attribute('id') References
b {'Congential heart disease': {'Symptoms': '', 'Diagnosis': '', 'RiskFactors': '', 'Prevention': ''}, 'Peripheral vascular disease': {'Symptoms': 'The signs and symptoms of peripheral artery di

element.get_attribute('id') Classification
element.get_attribute('id') Berry_.28saccular.29_aneurysms
element.get_attribute('id') Berry_(saccular)_aneurysms
element.get_attribute('id') Fusiform_aneurysms
element.get_attribute('id') Microaneurysms
element.get_attribute('id') Signs_and_symptoms
checking symptoms
p A small, unchanging aneurysm will produce few, if any, symptoms. Before a larger aneurysm ruptures, the individual may experience such symptoms as a sudden and unusually severe headache, nausea, vision impairment, vomiting, and loss of consciousness, or no symptoms at all.[10]
in get_symptoms
in get_paragraph
element.get_attribute('id') Subarachnoid_bleed
element.get_attribute('id') Microaneurysms_2
element.get_attribute('id') Vasospasm
element.get_attribute('id') Risk_factors
in getRiskFactors
in get_paragraph
element.get_attribute('id') Genetic_associations
element.get_attribute('id') Pathophysiology
element.get_attribute('id') Saccular_aneurysms
element.get_attribute('id') D

...

element.get_attribute('id') Signs_and_symptoms
checking symptoms
p Lower urinary tract infection is also referred to as a bladder infection. The most common symptoms are burning with urination and having to urinate frequently (or an urge to urinate) in the absence of vaginal discharge and significant pain.[4] These symptoms may vary from mild to severe[10] and in healthy women last an average of six days.[19] Some pain above the pubic bone or in the lower back may be present. People experiencing an upper urinary tract infection, or pyelonephritis, may experience flank pain, fever, or nausea and vomiting in addition to the classic symptoms of a lower urinary tract infection.[10] Rarely, the urine may appear bloody[7] or contain visible pus in the urine.[21]
in get_symptoms
in get_paragraph
element.get_attribute('id') Children
element.get_attribute('id') Elderly
element.get_attribute('id') Cause
element.get_attribute('id') Intercourse
element.get_attribute('id') Sex
element.get_attribute

a {'Congential heart disease': {'Symptoms': '', 'Diagnosis': '', 'RiskFactors': '', 'Prevention': ''}, 'Peripheral vascular disease': {'Symptoms': 'The signs and symptoms of peripheral artery disease are based on the part of the body that is affected. About 66% of patients affected by PAD either do not have symptoms or have atypical symptoms.[19] The most common presenting symptom is intermittent claudication (IC), which typically refers to lower extremity skeletal muscle pain that occurs during exercise. IC presents when there is insufficient oxygen delivery to meet the metabolic requirements of the skeletal muscles. IC is a common manifestation of peripheral arterial disease (PAD). The pain is usually located in the calf muscles in the affected leg and relieved by rest.[30] This occurs because during exercise the muscles require more oxygen. Normally, the arteries would be able to increase the amount of blood flow and therefore increase the amount of oxygen going to the exercised mus

element.get_attribute('id') Signs_and_symptoms
checking symptoms
p Some or all of the following symptoms may be present, though it is possible not to experience any symptoms:[4]
in get_symptoms
in get_paragraph
element.get_attribute('id') Cyst_rupture
element.get_attribute('id') Ovarian_torsion
element.get_attribute('id') Types
element.get_attribute('id') Functional
element.get_attribute('id') Non-functional
element.get_attribute('id') Diagnosis
checking diagnosis
in getDiagnonis
in get_paragraph
element.get_attribute('id') Ultrasound
element.get_attribute('id') Scoring_systems
element.get_attribute('id') Associated_conditions
element.get_attribute('id') Risk_of_cancer
element.get_attribute('id') Histopathology
element.get_attribute('id') Treatment
element.get_attribute('id') Pain
element.get_attribute('id') Surgery
element.get_attribute('id') Frequency
element.get_attribute('id') References
b {'Congential heart disease': {'Symptoms': '', 'Diagnosis': '', 'RiskFactors': '', 'Prevention

a {'Congential heart disease': {'Symptoms': '', 'Diagnosis': '', 'RiskFactors': '', 'Prevention': ''}, 'Peripheral vascular disease': {'Symptoms': 'The signs and symptoms of peripheral artery disease are based on the part of the body that is affected. About 66% of patients affected by PAD either do not have symptoms or have atypical symptoms.[19] The most common presenting symptom is intermittent claudication (IC), which typically refers to lower extremity skeletal muscle pain that occurs during exercise. IC presents when there is insufficient oxygen delivery to meet the metabolic requirements of the skeletal muscles. IC is a common manifestation of peripheral arterial disease (PAD). The pain is usually located in the calf muscles in the affected leg and relieved by rest.[30] This occurs because during exercise the muscles require more oxygen. Normally, the arteries would be able to increase the amount of blood flow and therefore increase the amount of oxygen going to the exercised mus

element.get_attribute('id') Etymology
element.get_attribute('id') Signs_and_symptoms
checking symptoms
p The period between infection and the first symptoms (incubation period) is typically one—three months in humans.[22] This period may be as short as four days or longer than six years, depending on the location and severity of the wound and the amount of virus introduced.[22] Initial symptoms of rabies are often nonspecific such as fever and headache.[22] As rabies progresses and causes inflammation of the brain and meninges, symptoms can include slight or partial paralysis, anxiety, insomnia, confusion, agitation, abnormal behavior, paranoia, terror, and hallucinations.[9][22] The person may also have fear of water.[1]
in get_symptoms
in get_paragraph
element.get_attribute('id') Cause
element.get_attribute('id') Transmission
element.get_attribute('id') Diagnosis
checking diagnosis
in getDiagnonis
in get_paragraph
element.get_attribute('id') Prevention
in getPrevention
in get_paragra

In [544]:
data_comfirmation_dictionary

{'Coronary heart disease': 'has data'}

In [546]:
disease_details.keys()

dict_keys(['Congential heart disease', 'Coronary heart disease'])

In [287]:
disease_details['Coronary heart disease']['Coronary heart disease']

{'Symptoms': 'The most common symptom is chest pain or discomfort that occurs regularly with activity, after eating, or at other predictable times; this phenomenon is termed stable angina and is associated with narrowing of the arteries of the heart. Angina also includes chest tightness, heaviness, pressure, numbness, fullness, or squeezing.[27] Angina that changes in intensity, character or frequency is termed unstable. Unstable angina may precede myocardial infarction. In adults who go to the emergency department with an unclear cause of pain, about 30% have pain due to coronary artery disease.[28] Angina, shortness of breath, sweating, nausea or vomiting, and lightheadedness are signs of a heart attack, or myocardial infarction, and immediate emergency medical services are crucial.[27]With advanced disease, the narrowing of coronary arteries reduces the supply of oxygen-rich blood flowing to the heart, which becomes more pronounced during strenuous activities during which the heart 

In [93]:
disease_list

['Congential heart disease',
 'Coronary heart disease',
 'Atherosclerosis',
 'Cardiomyopathy',
 'Heart attack',
 'Pericarditis',
 'Peripheral vascular disease',
 'Rheumatic heart disease',
 'Aortic valve stenosis',
 'Congestive heart failure',
 'Atrial fibrillation',
 'Fatty liver',
 'Hepatitis',
 'Hepatitis A',
 'Hepatitis B',
 'Hepatitis C',
 'Hepatitis E',
 'Non alcoholic fatty liver disease',
 'Cirrhosis',
 'Chronic kidney disease',
 'Gallstones',
 'Kidney infection/Pyelonephritis',
 'Kidney stones',
 'Kidney failure',
 'Polycystic kidney disease',
 'Diabetic Nephropathy',
 'Nephrotic syndrome',
 'Glomerulonephritis',
 'Epilepsy',
 'Stroke',
 'Brain aneurysm',
 "Parkinson's disease",
 'Bipolar disorder',
 'Dementia',
 'Alzheimer’s disease',
 'Depression',
 'Schizophrenia',
 'Acidity',
 'Appendicitis',
 'Dyspepsia',
 'Food poisoning',
 'Gastritis',
 'Gastroenteritis',
 'IBS',
 'Peptic ulcer',
 'Colitis',
 'Food allergy',
 'Allergic rhinitis',
 'Asthma',
 'Cystic fibrosis',
 'Adenovi

In [564]:
disease_list1 = []
for i in  range(len(disease_list)):
    if (i%6) == 0.0:
        print(disease_list[i])
        disease_list1.append(disease_list[i])
    

Congential heart disease
Peripheral vascular disease
Hepatitis
Cirrhosis
Polycystic kidney disease
Brain aneurysm
Schizophrenia
Gastroenteritis
Asthma
Pneumonia
Tuberculosis
Glaucoma
Hyperopia or farsightedness
Thyroid disease
Diabetic neuropathy
Sciatica
Aneurysm
Osteomyelitits
Atopic dermatitis
Eczema
Gum disease
Breast cancer
Ovarian cancer
Lupus
Celiac disease
Ankylosing spondylitis
Multiple sclerosis
UTIs
Ovarian cysts
Rabies


In [553]:
"""elif 'risk factors' in el.get_attribute('id').lower():
                print(el.get_attribute('id'))
   elif 'prevention' in el.get_attribute('id').lower():
                print(el.get_attribute('id'))
        else:
            #print("passed")
            pass"""

'elif \'risk factors\' in el.get_attribute(\'id\').lower():\n                print(el.get_attribute(\'id\'))\n   elif \'prevention\' in el.get_attribute(\'id\').lower():\n                print(el.get_attribute(\'id\'))\n        else:\n            #print("passed")\n            pass'

In [567]:
len(disease_list)

30

In [566]:
disease_list  = disease_list1
